In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests
from PIL import Image
import time
import io

# function that scrolls height of webpage (to the very very bottom of the google images page)
def scroll_down(thedriver, delay):
        # scroll height of webpage
    last_height = thedriver.execute_script("return document.body.scrollHeight")
    while True:
        thedriver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # The driver scrolls down the webpage here.
        time.sleep(delay)
        new_height = thedriver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Breaks here if the driver reached the bottom of the webpage (when it cannot scroll down anymore).
            break
        last_height = new_height       # If the driver failed to scroll to the bottom of the webpage, the current scroll height is recorded. Following, it is compared to the scroll height in the next iteration of the loop to decide if the driver reached the bottom of the webpage or not.
    thedriver.execute_script("window.scrollTo(0, 0);")
    
def get_images(thedriver, max_images):
    
    image_urls = set()
    thumbnails = thedriver.find_elements(By.CLASS_NAME, "YQ4gaf")
    count = 0
    
    for img in thumbnails:
        
        if (int(img.get_attribute('height')) < 15):
            continue
        
        try:
            img.click()
            time.sleep(1.5)
        except:
            continue
        
        images = thedriver.find_elements(By.CLASS_NAME, "iPVvYb")
        for image in images:
            if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                count += 1
                last_size = len(image_urls)
                image_urls.add(image.get_attribute('src'))
                if (last_size != len(image_urls)):
                    print(len(image_urls), "URLS ADDED")
            if len(image_urls) >= max_images:
                return image_urls
            
    return image_urls

def download_image(download_path, url, file_name):
    try:
        image_content = requests.get(url).content
    except requests.exceptions.RequestException as e:
        print(f"Failed to download image from {url} - {e}")
        return # Return without attempting to process the image further
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
    except (IOError, OSError) as e:
        print(f"Failed to open image file - {e}")
        return # Return without attempting to process the image further
    try:
        # Convert image to RGB mode
        image = image.convert('RGB')
        file_path = download_path + file_name
        try:
            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
            print(f"Downloaded: {file_name}")
        except Exception as e:
            print(f"Failed to save image - {e}")
            return
    except Exception as e:
        print("Failed -", e)
        return
    return

def main(url, download_path):
    thedriver.get(url)
    scroll_down(thedriver, 3)
    time.sleep(2)
    image_urls = get_images(thedriver, 1000)
    print("GOT", len(image_urls), "IMAGE URLS")
    count = 0
    for link in image_urls:
        count += 1
        download_image(download_path, link, "image" + str(count) + ".jpg")
        if (len(thedriver.window_handles) > 1):
            thedriver.switch_to.window(thedriver.window_handles[1])
            thedriver.close()
            time.sleep(0.75)
            thedriver.switch_to.window(thedriver.window_handles[0])
            time.sleep(0.75)
    print("DOWNLOADED", count, "IMAGES")
    time.sleep(3)

thedriver = webdriver.Firefox()
url = "https://www.google.com/search?q=yellow+eyelash+viper&sca_esv=f2f47ebcac58f9dc&udm=2&biw=1408&bih=659&sxsrf=ADLYWIJGxzaN-SWCRMzzKGlnYqRxXppwaw%3A1732562131017&ei=08xEZ55b66LV8g-F1OapDA&ved=0ahUKEwjekPvymPiJAxVrUVUIHQWqOcUQ4dUDCBA&uact=5&oq=yellow+eyelash+viper&gs_lp=EgNpbWciFHllbGxvdyBleWVsYXNoIHZpcGVyMgQQIxgnMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBRAAGIAEMgoQABiABBhDGIoFSMQOULoIWM0NcAF4AJABAJgBwAKgAYwKqgEFMi00LjG4AQPIAQD4AQGYAgGgAguYAwCIBgGSBwExoAekJg&sclient=img"
main(url, "yellow_eyelash_viper\\")
url2 = "https://www.google.com/search?client=firefox-b-1-d&sca_esv=8c4a697f1b82db14&sxsrf=ADLYWILLDw3aY8LfW-zsAOIpi1hCHPw_7w:1732585473680&q=milk+snake&udm=2&fbs=AEQNm0Aa4sjWe7Rqy32pFwRj0UkWd8nbOJfsBGGB5IQQO6L3J03RPjGV0MznOJ6Likin94rtxCRB-KZnKMV4RyRovG_PDxn-EtieOVthDAs_X6rfui8V3nsI16ebR5rmwY_OA1KV3FMg9_QVrSgNG6ly6PHGduuPc6dL_vs6tZJKHMXJeia9LfrP88I3EVfFJQPK92dXUUziEsdsXCYqH3UNRlzaKi_Z2g&sa=X&ved=2ahUKEwj_n87t7_iJAxWIRvEDHddbMHUQtKgLegQIHxAB&biw=1408&bih=659&dpr=1.82#vhid=B95-KO9X87dkFM&vssid=mosaic"
main(url2, "milk_snake\\")
url3 = "https://www.google.com/search?client=firefox-b-1-d&sca_esv=a67a64356a29d00d&sxsrf=ADLYWIIXXfWUJK_SLlYZC-x7P2aStZgrww:1732588482035&q=ribbon+snake&udm=2&fbs=AEQNm0Aa4sjWe7Rqy32pFwRj0UkWd8nbOJfsBGGB5IQQO6L3J03RPjGV0MznOJ6Likin94o1r5LfEXWFkRDyPajU30-jlDcXwRVTByGx4VdKZ6GPzFySlTPBGOjLRBdHTKxeZmdeIA02KMtcmXYbZ9s2Kwl2xj5KMk3FW13OuMR1VH1zYgWAk0ArXQEf7T3DGJM9HnfG-20ubQhN1hlsmfo2u2xXJCINMg&sa=X&ved=2ahUKEwjS0o2I-_iJAxWsHxAIHUmBFUsQtKgLegQIMBAB&biw=1408&bih=659&dpr=1.82"
main(url3, "ribbon_snake\\")
thedriver.quit()
print("CLOSED DRIVER")